# Azure Container Apps dynamic sessions

Azure Container Apps dynamic sessions provides a secure and scalable way to run a Python code interpreter in Hyper-V isolated sandboxes. This allows your agents to run potentially untrusted code in a secure environment. The code interpreter environment includes many popular Python packages, such as NumPy, pandas, and scikit-learn.

## Pre-requisites

By default, the `SessionsPythonREPLTool` tool uses `DefaultAzureCredential` to authenticate with Azure. Locally, it'll use your credentials from the Azure CLI or VS Code. Install the Azure CLI and log in with `az login` to authenticate.

## Using the tool

Set variables:

In [1]:
import os
import dotenv
dotenv.load_dotenv()

POOL_MANAGEMENT_ENDPOINT = os.getenv("POOL_MANAGEMENT_ENDPOINT")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

In [4]:
from langchain_azure_dynamic_sessions import SessionsPythonREPLTool


tool = SessionsPythonREPLTool(pool_management_endpoint=POOL_MANAGEMENT_ENDPOINT)
tool.run("6 * 7")


'{\n  "result": 42,\n  "stdout": "",\n  "stderr": ""\n}'

## Full agent example

In [3]:
import os
from azure.identity import DefaultAzureCredential
from langchain_azure_dynamic_sessions import SessionsPythonREPLTool
from langchain_openai import AzureChatOpenAI
from langchain import agents, hub


credential = DefaultAzureCredential()
os.environ["OPENAI_API_TYPE"] = "azure_ad"
os.environ["OPENAI_API_KEY"] = credential.get_token("https://cognitiveservices.azure.com/.default").token
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",
    openai_api_version="2023-09-15-preview",
    streaming=True,
    temperature=0,
)

repl = SessionsPythonREPLTool(
    pool_management_endpoint=POOL_MANAGEMENT_ENDPOINT,
)

tools = [repl]
react_agent = agents.create_react_agent(
    llm=llm,
    tools=tools,
    prompt=hub.pull("hwchase17/react"),
)

react_agent_executor = agents.AgentExecutor(agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True)

react_agent_executor.invoke({"input": "If I put $500 in a bank account with a 5% interest rate, how much money will I have in the account after 6 years?"})



> Entering new AgentExecutor chain...
I need to calculate the compound interest on the initial amount over 6 years.
Action: Python_REPL
Action Input: 
```python
initial_amount = 500
interest_rate = 0.05
time_period = 6
final_amount = initial_amount * (1 + interest_rate)**time_period
final_amount
```{
  "result": 670.0478203125002,
  "stdout": "",
  "stderr": ""
}The final amount after 6 years will be $670.05
Final Answer: $670.05

> Finished chain.


{'input': 'If I put $500 in a bank account with a 5% interest rate, how much money will I have in the account after 6 years?',
 'output': '$670.05'}